<a href="https://colab.research.google.com/github/MatienkoAndrew/notebooks/blob/main/%5Bhomework%5Dembeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

# Embeddings

Привет! В этом домашнем задании мы с помощью эмбеддингов решим задачу семантической классификации твитов.

Для этого мы воспользуемся предобученными эмбеддингами word2vec.

Для начала скачаем датасет для семантической классификации твитов:

In [ ]:
# !gdown https://drive.google.com/uc?id=1eE1FiUkXkcbw0McId4i7qY-L8hH-_Qph&export=download
!unzip archive.zip

Импортируем нужные библиотеки:

## Libs

In [ ]:
import math
import random
import string

import numpy as np
import pandas as pd
import seaborn as sns

import torch
# import nltk
# import gensim
# import gensim.downloader as api

In [ ]:
random.seed(42)
np.random.seed(42)
torch.random.manual_seed(42)
torch.cuda.random.manual_seed(42)
torch.cuda.random.manual_seed_all(42)

device = "cuda" if torch.cuda.is_available() else "cpu"

## Data

In [ ]:
data = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding="latin", header=None, names=["emotion", "id", "date", "flag", "user", "text"])

In [ ]:
data.info()

Посмотрим на данные:

In [ ]:
data

In [ ]:
data['emotion'].value_counts()

Выведем несколько примеров твитов, чтобы понимать, с чем мы имеем дело:

In [ ]:
examples = data['text'].sample(10)
print('\n'.join(examples))
# examples = data["text"].sample(10)
# print("\n".join(examples))

Как вилим, тексты твитов очень "грязные". Нужно предобработать датасет, прежде чем строить для него модель классификации.

Чтобы сравнивать различные методы обработки текста/модели/прочее, разделим датасет на dev(для обучения модели) и test(для получения качества модели).

## Train test split

In [ ]:
np.random.seed(42)

indexes = np.arange(data.shape[0])
np.random.shuffle(indexes)
dev_size = math.ceil(data.shape[0] * 0.8)

dev_indexes = indexes[:dev_size]
test_indexes = indexes[dev_size:]

dev_data = data.iloc[dev_indexes]
test_data = data.iloc[test_indexes]

dev_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

## Обработка текста

Токенизируем текст, избавимся от знаков пунктуации и выкинем все слова, состоящие менее чем из 4 букв:

In [ ]:
import nltk

tokenizer = nltk.WordPunctTokenizer()
line = tokenizer.tokenize(dev_data["text"][0].lower())
print('Before:')
print(dev_data['text'][0])
print('After')
print(" ".join(line))

# tokenizer = nltk.WordPunctTokenizer()
# line = tokenizer.tokenize(dev_data["text"][0].lower())
# print(" ".join(line))

In [ ]:
filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
print(' '.join(filtered_line))
# filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
# print(" ".join(filtered_line))

Загрузим предобученную модель эмбеддингов. 

Если хотите, можно попробовать другую. Полный список можно найти здесь: https://github.com/RaRe-Technologies/gensim-data.

Данная модель выдает эмбеддинги для **слов**. Строить по эмбеддингам слов эмбеддинги предложений мы будем ниже.

## word2vec

In [ ]:
import gensim
import gensim.downloader as api

word2vec = api.load("word2vec-google-news-300")

In [ ]:
emb_line = [word2vec.get_vector(w) for w in filtered_line if w in word2vec]
print(sum(emb_line).shape)

# emb_line = [word2vec.get_vector(w) for w in filtered_line if w in word2vec]
# print(sum(emb_line).shape)

Нормализуем эмбеддинги, прежде чем обучать на них сеть. 
(наверное, вы помните, что нейронные сети гораздо лучше обучаются на нормализованных данных)

## Normalize embeddings

In [ ]:
mean = np.mean(word2vec.vectors, 0)
std = np.std(word2vec.vectors, 0)
norm_emb_line = [(word2vec.get_vector(w) - mean) / std for w in filtered_line if w in word2vec and len(w) > 3]
print(sum(norm_emb_line).shape)
print([all(norm_emb_line[i] == emb_line[i]) for i in range(len(emb_line))])

# mean = np.mean(word2vec.vectors, 0)
# std = np.std(word2vec.vectors, 0)
# norm_emb_line = [(word2vec.get_vector(w) - mean) / std for w in filtered_line if w in word2vec and len(w) > 3]
# print(sum(norm_emb_line).shape)
# print([all(norm_emb_line[i] == emb_line[i]) for i in range(len(emb_line))])

Сделаем датасет, который будет по запросу возвращать подготовленные данные.

In [ ]:
from torch.utils.data import Dataset, random_split


class TwitterDataset(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        # Получи все токены из текста и профильтруй их
        tokenizer = self.tokenizer
        return [w for w in tokenizer.tokenize(text.lower()) if all(c not in string.punctuation for c in w) and len(w) > 3]

    def get_embeddings_(self, tokens):
        # Получи эмбеддинги слов и усредни их
        word2vec = self.word2vec
        embeddings = [(word2vec.get_vector(w) - mean) / std for w in tokens if w in word2vec and len(w) > 3]

        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [ ]:
%%time
dev = TwitterDataset(dev_data, "text", "emotion", word2vec)

In [ ]:
##-- test
# dev_data.iloc[4]
# dev_data.loc[4, 'text']
# dev.get_tokens_(dev_data.loc[4, 'text'])
# dev[4]['feature'].shape

Отлично, мы готовы с помощью эмбеддингов слов превращать твиты в векторы и обучать нейронную сеть.

Превращать твиты в векторы, используя эмбеддинги слов, можно несколькими способами. А именно такими:

## Average embedding (2 балла)
---
Это самый простой вариант, как получить вектор предложения, используя векторные представления слов в предложении. А именно: вектор предложения есть средний вектор всех слов в предлоежнии (которые остались после токенизации и удаления коротких слов, конечно). 

In [ ]:
indexes = np.arange(len(dev))
np.random.shuffle(indexes)
example_indexes = indexes[::1000]

examples = {"features": [np.mean(dev[i]["feature"], axis=0) for i in example_indexes], 
            "targets": [dev[i]["target"] for i in example_indexes]}
print(len(examples["features"]))

Давайте сделаем визуализацию полученных векторов твитов тренировочного (dev) датасета. Так мы увидим, насколько хорошо твиты с разными target значениями отделяются друг от друга, т.е. насколько хорошо усреднение эмбеддингов слов предложения передает информацию о предложении.

Для визуализации векторов надо получить их проекцию на плоскость. Сделаем это с помощью `PCA`. Если хотите, можете вместо PCA использовать TSNE: так у вас получится более точная проекция на плоскость (а значит, более информативная, т.е. отражающая реальное положение векторов твитов в пространстве). Но TSNE будет работать намного дольше.

In [ ]:
from sklearn.decomposition import PCA


pca = PCA(n_components=2)
examples["transformed_features"] = pca.fit_transform(examples['features']) # Обучи PCA на эмбеддингах слов

In [ ]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

In [ ]:
draw_vectors(
    examples["transformed_features"][:, 0], 
    examples["transformed_features"][:, 1], 
    color=[["red", "blue"][t] for t in examples["targets"]]
    )

Скорее всего, на визуализации нет четкого разделения твитов между классами. Это значит, что по полученным нами векторам твитов не так-то просто определить, к какому классу твит пренадлежит. Значит, обычный линейный классификатор не очень хорошо справится с задачей. Надо будет делать глубокую (хотя бы два слоя) нейронную сеть.

Подготовим загрузчики данных.
Усреднее векторов будем делать в "батчевалке"(`collate_fn`). Она используется для того, чтобы собирать из данных `torch.Tensor` батчи, которые можно отправлять в модель.


## Dataloader

In [ ]:
from torch.utils.data import DataLoader

batch_size = 1024
num_workers = 8

def average_emb(batch):
    features = np.array([np.mean(b['feature'], axis=0) for b in batch])
    targets = np.array([b['target'] for b in batch])
    return {"features": torch.FloatTensor(features), "targets": torch.LongTensor(targets)}
    # return {"features": torch.FloatTensor(features), "targets": torch.LongTensor(targets)} ##-- too slow

train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

Определим функции для тренировки и теста модели:

## Train test funcs

In [ ]:
from tqdm.notebook import tqdm
from tqdm import tqdm

def training(model, optimizer, criterion, train_loader, epoch, device="cpu"):
    mean_loss = 0
    loss_sum = 0
    pbar = tqdm(train_loader, desc=f"Epoch {e + 1}. Train Loss: {0}")
    model.train()
    for batch in pbar:
        features = batch["features"].to(device)
        targets = batch["targets"].to(device)

        predictions_prob = model(features) # Получи предсказания модели
        # print("Predictions:", predictions.dtype, predictions.shape)
        # print("Targets:", targets.dtype, targets.shape)
        loss = criterion( predictions_prob, targets ) # Посчитай лосс
        loss.backward() # Расчет градиентов
        optimizer.step() # Обнови параметры модели

        pbar.set_description(f"Epoch {e + 1}. Train Loss: {loss:.4}")

        loss_sum += loss.item()
        mean_loss = loss_sum / len(train_loader)
        pbar.set_description(f"Mean Loss: {mean_loss:.4}")
    return {"Train Loss": mean_loss}

def testing(model, criterion, test_loader, device="cpu"):
    pbar = tqdm(test_loader, desc=f"Test Loss: {0}, Test Acc: {0}")
    mean_loss = 0
    mean_acc = 0 
    model.eval()
    with torch.no_grad():
        for batch in pbar:
            features = batch['features'].to(device)
            targets = batch['targets'].to(device)

            # Получи предсказания модели
            predictions_prob = model(features)
            # print("predictions_prob:", predictions_prob, predictions_prob.dtype, predictions_prob.shape)
            # print("Targets:", targets, targets.dtype, targets.shape)
            loss = criterion( predictions_prob, targets ) # Посчитай лосс

            predictions = torch.argmax(predictions_prob, dim=-1) # переводим предсказания модели в классы
            # print("predictions:", predictions, predictions.dtype, predictions.shape)

            # predictions = (predictions_prob >= 0.5).int()
            acc = torch.sum(predictions == targets) / predictions.shape[0] # Посчитай точность модели

            mean_loss += loss.item()
            mean_acc += acc.item()

            pbar.set_description(f"Test Loss: {loss:.4}, Test Acc: {acc:.4}")

    pbar.set_description(f"Test Loss: {mean_loss / len(test_loader):.4}, Test Acc: {mean_acc / len(test_loader):.4}")

    return {"Test Loss": mean_loss / len(test_loader), "Test Acc": mean_acc / len(test_loader)}

"""
def training(model, optimizer, criterion, train_loader, epoch, device="cpu"):
    pbar = tqdm(train_loader, desc=f"Epoch {e + 1}. Train Loss: {0}")
    model.train()
    for batch in pbar:
        features = batch["features"].to(device)
        targets = batch["targets"].to(device)

        # Получи предсказания модели
        loss = ... # Посчитай лосс
        # Обнови параметры модели

        pbar.set_description(f"Epoch {e + 1}. Train Loss: {loss:.4}")
    

def testing(model, criterion, test_loader, device="cpu"):
    pbar = tqdm(test_loader, desc=f"Test Loss: {0}, Test Acc: {0}")
    mean_loss = 0
    mean_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in pbar:
            features = batch["features"].to(device)
            targets = batch["targets"].to(device)

            # Получи предсказания модели
            loss = ... # Посчитай лосс
            acc = ... # Посчитай точность модели

            mean_loss += loss.item()
            mean_acc += acc.item()

            pbar.set_description(f"Test Loss: {loss:.4}, Test Acc: {acc:.4}")

    pbar.set_description(f"Test Loss: {mean_loss / len(test_loader):.4}, Test Acc: {mean_acc / len(test_loader):.4}")

    return {"Test Loss": mean_loss / len(test_loader), "Test Acc": mean_acc / len(test_loader)}
"""

Создадим модель, оптимизатор и целевую функцию. Вы можете сами выбрать количество слоев в нейронной сети, ваш любимый оптимизатор и целевую функцию.


## Classifier

In [ ]:
import torch.nn as nn

class Classifier(nn.Module):
    def __init__(
            self, features_num, classes_num=2,
            layer_neurons_num=256, layers_num=2, bias=True,
            batchnorm=False, batchnorm_before_activation=False):
        if layers_num < 1:
            raise ValueError
        super().__init__()
        layers = [nn.Linear(in_features=features_num, out_features=layer_neurons_num if layers_num > 1 else classes_num)]
        if layers_num - 1 > 0:
            for _ in range(layers_num - 2):
                a = nn.ReLU(inplace=True)
                if batchnorm:
                    b = nn.BatchNorm1d(layer_neurons_num)
                    layers.extend((b, a) if batchnorm_before_activation else (a, b))
                else:
                    layers.append(a)
                layers.append(nn.Linear(layer_neurons_num, layer_neurons_num, bias=bias))
            # We do not need batch normalization before the output layer
            # (there is opinion it may make worse classification results).
            layers.extend([nn.ReLU(inplace=True), nn.Linear(layer_neurons_num, classes_num, bias=bias)])
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x).squeeze()
    
    def binary_predict(input, output_type=torch.long):
        return (torch.sigmoid(input) >= 0.5).type(output_type)

## Hyperparams

In [ ]:
from torch.optim import Adam

vector_size = dev.word2vec.vector_size
num_classes = 2
lr = 1e-2
num_epochs = 1


default_model_init_kwargs = dict(
    features_num=word2vec.vector_size, classes_num=2,
    layer_neurons_num=256, layers_num=2, bias=True,
    batchnorm=False, batchnorm_before_activation=False)
model = Classifier(**default_model_init_kwargs)
model = model.to(device)
# criterion = nn.BCEWithLogitsLoss()
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=lr)

Наконец, обучим модель и протестируем её.

После каждой эпохи будем проверять качество модели на валидационной части датасета. Если метрика стала лучше, будем сохранять модель. **Подумайте, какая метрика (точность или лосс) будет лучше работать в этой задаче?** 

## Fit

In [ ]:
##-- for test 
# train_loader = DataLoader(examples, batch_size=batch_size, num_workers=2, shuffle=True, drop_last=True, collate_fn=average_emb)

In [ ]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]

## Metrics

In [ ]:
test_loader = DataLoader(
    TwitterDataset(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))

## Embeddings for unknown words (8 баллов)

Пока что использовалась не вся информация из текста. Часть информации фильтровалось – если слова не было в словаре эмбеддингов, то мы просто превращали слово в нулевой вектор. Хочется использовать информацию по-максимуму. Поэтому рассмотрим другие способы обработки слов, которых нет в словаре. А именно:

- Для каждого незнакомого слова будем запоминать его контекст(слова слева и справа от этого слова). Эмбеддингом нашего незнакомого слова будет сумма эмбеддингов всех слов из его контекста. (4 балла)
- Для каждого слова текста получим его эмбеддинг из Tfidf с помощью ```TfidfVectorizer``` из [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer). Итоговым эмбеддингом для каждого слова будет сумма двух эмбеддингов: предобученного и Tfidf-ного. Для слов, которых нет в словаре предобученных эмбеддингов, результирующий эмбеддинг будет просто полученный из Tfidf. (4 балла)

Реализуйте оба варианта **ниже**. Напишите, какой способ сработал лучше и ваши мысли, почему так получилось.